In [1]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [2]:
def mat_dbg(x):
    print(x.shape, ": \n", x)

In [46]:
class MLP(object):
    """
    NOTE: Matrix operations are modified from RBM file
    In particular, we have an an an input of n data points with
    dimension d as a d-by-n matrix
    """
    def __init__(self, data, num_hidden, learn_rate):
        # First row are the target values
        self.targets = data[0]
        self.data = data[1:,:]
        self.num_data = data.shape[1]
        self.num_visible = data.shape[0] - 1
        self.num_hidden = num_hidden
        self.learn_rate = learn_rate
        
        self.min_data = np.amin(self.targets)
        self.max_data = np.amax(self.targets)
        self.state_size = 2 * (self.max_data - self.min_data) + 4
        
        self.weights = np.random.rand(self.num_visible + 1, self.num_hidden)
        
        self.out_layer = self.state_size * np.ones((self.num_hidden,1))
        
        self.data = np.insert(self.data, 0, 1, axis = 0)
    
    def _sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
        
    def _activated(self,mat):
        activated = self._sigmoid(np.dot(self.weights.T,mat))
        return activated
    
    def _gradient(self,mat):
        w = copy.deepcopy(mat)
        for elt in np.nditer(w,op_flags=['readwrite']):
            elt[...] = elt - elt * elt
        return w
    
    def _feed_forward(self):
        data_copy = copy.deepcopy(self.data)
        for i in range(0,self.num_data):
            data_col = data_copy[:,i]
            data_col = np.reshape(data_col, (-1, 1))
            
            activated_output = (self._activated(data_col))[0]
            guess = self.state_size * (2 * activated_output - 1)
            #print("Guess: ", guess)
            
            error = self.targets[i] - guess
            grad = (self._gradient(activated_output))[0]
            
            c = (error * self.learn_rate)[0]
            wt_change = c * grad * data_col
            
            self.weights = wt_change + self.weights
            self.weights = self.weights
        
        #print("New weights: ",self.weights)
        
    def _prop_backward(self):
        return None
    
    def _calc_error(self, mat):
        return None
    
    def pred(self, test):
        test_copy = copy.deepcopy(test)
        test_copy = np.insert(test_copy, 0, 1, axis = 0)
        pred_p = self._activated(test_copy)
        pred = 2 * pred_p - 1
        pred = self.state_size * pred
        return pred


In [47]:
# See www.cse.unsw.edu.au/~cs9417ml/MLP2/
# https://www.hiit.fi/u/ahonkela/dippa/node41.html
dims = 4
data_pts = 300 * dims
sample_data = np.sign(2*np.random.rand(dims,data_pts)-1)
sample_data[0] = (2 * sample_data[1] + 1 * sample_data[2]) 
# Need to write binary target values for data as well for MLP
mat_dbg(sample_data)
MLP_1 = MLP(sample_data,2,0.01)
print("Weights^T: \n",MLP_1.weights.T)
print("Targets: ")
print(MLP_1.targets)
print("Data sample: \n",MLP_1.data[1:5])

(4, 1200) : 
 [[-3.  1.  3. ...,  3.  3.  1.]
 [-1.  1.  1. ...,  1.  1.  1.]
 [-1. -1.  1. ...,  1.  1. -1.]
 [-1.  1.  1. ...,  1. -1.  1.]]
Weights^T: 
 [[ 0.39890333  0.75967996  0.81475514  0.53170743]
 [ 0.85632306  0.10813995  0.44291001  0.29674713]]
Targets: 
[-3.  1.  3. ...,  3.  3.  1.]
Data sample: 
 [[-1.  1.  1. ...,  1.  1.  1.]
 [-1. -1.  1. ...,  1.  1. -1.]
 [-1.  1.  1. ...,  1. -1.  1.]]


In [48]:
iterations = 2
for i in range(0,iterations):
    MLP_1._feed_forward()
#MLP_1._feed_forward()
print("Weights: \n", MLP_1.weights)
results = np.dot(MLP_1.weights.T,MLP_1.data)
print("Targets: \n", MLP_1.targets)
print("Results: \n", results)

Weights: 
 [[  5.69699934e-17   4.57419726e-01]
 [  2.52326382e-01  -3.99213622e-01]
 [  1.27163239e-01  -2.44681889e-01]
 [ -4.06668507e-19  -2.34960299e-01]]
Targets: 
 [-3.  1.  3. ...,  3.  3.  1.]
Results: 
 [[-0.37948962  0.12516314  0.37948962 ...,  0.37948962  0.37948962
   0.12516314]
 [ 1.33627554  0.06792769 -0.42143608 ..., -0.42143608  0.04848451
   0.06792769]]


In [49]:
dims = 3
test_pts = 10
test_data = np.sign(2*np.random.rand(dims,test_pts)-1)
print(test_data)
print(2 * test_data[0] + 1 * test_data[1])
a = MLP_1.pred(test_data)
print("Shape:",np.shape(a))
a

[[-1. -1.  1. -1. -1. -1.  1.  1.  1.  1.]
 [ 1. -1. -1.  1. -1.  1. -1. -1.  1. -1.]
 [-1.  1.  1. -1. -1. -1. -1. -1.  1. -1.]]
[-1. -3.  1. -1. -3. -1.  1.  1.  3.  1.]
Shape: (2, 10)


array([[-1.        , -3.        ,  1.        , -1.        , -3.        ,
        -1.        ,  1.        ,  1.        ,  3.        ,  1.        ],
       [ 6.39740519,  6.52757346,  0.54321269,  6.39740519,  9.34005719,
         6.39740519,  4.20197547,  4.20197547, -3.32245891,  4.20197547]])